In [ ]:
import time
from kafka import SimpleProducer,KafkaClient

#模拟生产者
#连接kafka
kafka = KafkaClient('localhost:9092')
proceducer = SimpleProducer(kafka)
topic = 'my-topic'

def test_log():
    print('begin')
    n = 1
    while (n < 100):
        proceducer.send_messages(topic,str(n))
        print('send' + str(n))
        n += 1
        time.sleep(0.5)
    print('done')

if __name__ == '__main__':
    test_log()

In [ ]:
#创建消费者
from kafka import KafkaConsumer

#连接topic
consumer = KafkaConsumer('my-topic')
print('begin')
for msg in consumer:
    print(msg)
    

In [16]:
w = '12wo23 r d3 5 '
w.strip()

'12wo23 r d3 5'

In [17]:
w.strip('wo23')

'12wo23 r d3 5 '

In [21]:
import cv2
import numpy as np
import math

In [22]:
import tensorflow as tf

In [40]:
import requests
def geocode(location):
        parameters = {'location':location,'key':'c69bd09bde2e2a17eff5edf9b89720a5'}
        base = 'https://restapi.amap.com/v3/geocode/regeo'
        response = requests.get(base, parameters)
        answer = response.json()
        print(answer['info'])

location = (118.903882,32.096447)
geocode(location)


INVALID_PARAMS


In [52]:
import requests

def geocode(address):
    parameters = {'address': address, 'key': '38c8342d5a1a44ea3d1c488627eb3742'}
    base = 'https://restapi.amap.com/v3/geocode/geo'
    response = requests.get(base, parameters)
    answer = response.json()
    print(address + "的经纬度：", answer['geocodes'])

if __name__=='__main__':
    #address = input("请输入地址:")
    address = '北京市海淀区'
    geocode(address)

北京市海淀区的经纬度： [{'formatted_address': '北京市海淀区', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '海淀区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110108', 'street': [], 'number': [], 'location': '116.329519,39.972134', 'level': '区县'}]


In [54]:
import requests

def regeocode(address):
    parameters = {'address': address, 'key': '38c8342d5a1a44ea3d1c488627eb3742'}
    base = 'https://restapi.amap.com/v3/geocode/regeo'
    response = requests.get(base, parameters)
    answer = response.json()
    print(answer['info'])

if __name__=='__main__':
    #address = input("请输入地址:")
    address = (116.481488,39.990464)
    regeocode(address)

INVALID_PARAMS


In [43]:
# select addr from place where lat > lat1 and lat < lat2 and lng > lng1 and lng < lng2 limit 10
import pymysql
db = pymysql.connect(host='47.102.28.202', port=3306, user='readonly', password='abc123$%',
                     database='renwochong', charset='utf8')
cursor = db.cursor()
select_cord = """select addr from t_charging_station where latitude > 32.10 and latitude < 32.25 and longitude > 118.50 and longitude < 119.20"""

#执行相关查询
try:
    res = cursor.execute(select_cord)
    res = cursor.fetchall()
    print(res)
except Exception as e:
    print(e)

    print(type(corareas1))
    res1 = pd.DataFrame(corareas1)
    print(res1)



(('南京市江北新区浦六南路988号',), ('江苏省南京市栖霞区乐和酒店（聚宝山店）',))


In [1]:
import time

In [4]:
print(time.strftime('%Y%M%D'))

20192904/09/19


In [6]:
from datetime import datetime

In [4]:
#获取当前日期并做转换
print(datetime.datetime.now())
print('---------')
time_now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
print(time_now)
# int_time = int(time_now)
# print(type(int_time))
# print(int_time)

2019-04-10 09:56:35.475196
---------
20190410095635


In [ ]:
datetime.datetime.strptime(string,'%Y-%m-%d %H:%M:%S')

In [8]:
print(datetime.now().strftime('%Y%m%d%H'))

2019041010


In [16]:
time_day = datetime.now().strftime('%Y%m%d')
file = lambda x:x[-4:]=='.log' and x[0:8]==time_day
file('2019041010.log')

True

In [ ]:
#!/usr/bin/env python
#--*-- coding:utf-8 --*--

#python根据一个给定经纬度的点，进行附近5km距离坐标查询
import pymysql
import os
import subprocess
import time
import numpy as np
import pandas as pd
from math import sin, asin, cos, radians, fabs, sqrt,degrees

EARTH_RADIUS=6371 #地球半径


#读取车辆坐标位置文件(静态文件读取)
def get_cortxt():
    df = pd.read_csv('D:/cor.txt',encoding='utf-8',header=None,sep=' ')
    for i in range(len(df)):
        coordinate = df.loc[i]
        cor_areas(coordinate)


#计算坐标点周围5km，东西两侧距离,lat表示纬度，lng表示经度
def cor_areas(coordinate):
    dlng = 2 * asin(sin(distance / (2 * EARTH_RADIUS)) / cos(coordinate[1]))
    dlng = degrees(dlng)        # 弧度转换成角度

    #计算坐标点周围5km，南北两侧距离坐标
    dlat = distance / EARTH_RADIUS
    dlat = degrees(dlat)     # 弧度转换成角度
    
    #cor1左上角坐标点，cor2右下角坐标点
    cor1_vin,cor1_lat,cor1_lng = coordinate[0],coordinate[1] + dlat,coordinate[2] - dlng
    cor2_vin,cor2_lat,cor2_lng = coordinate[0],coordinate[1] - dlat,coordinate[2] + dlng

    mysql_query(cor1_lat,cor1_lng,cor2_lat,cor2_lng)
    #     mysql_query(32.006447,118.003882,31.096447,118.903882)


#将车辆坐标传入mysql数据库查询附近充电站的位置信息
def mysql_query(cor1_lat,cor1_lng,cor2_lat,cor2_lng):
        # sql中查询在此坐标5km范围内的充电站坐标
        import pymysql
        db = pymysql.connect(host='106.15.223.235', port=3306, user='readonly', password='abc123$%',
                             database='renwochong', charset='utf8')
        cursor = db.cursor()
#         select_cord = """select addr from t_charging_station where latitude > cor2_lat and latitude < cor1_lat and longitude > cor1_lng and longitude < cor2_lng"""
        select_cord = """select addr from t_charging_station where latitude > %f and latitude < %f and longitude > %f and longitude < %f"""%(cor2_lat,cor1_lat,cor1_lng,cor2_lng)
        #执行相关查询
        try:
            res = cursor.execute(select_cord)
            res = cursor.fetchall()
            print(res)
        except Exception as e:
            print(e)

#主函数部分
if __name__ == '__main__':

    distance = 5  #车辆周围距离5km
    get_cortxt()


